In [1]:
from keras.datasets import cifar10
import keras
import keras.backend as K
import numpy as np
from datetime import datetime
from data_utils import CIFAR10DataLoader
from network import GetPrediction,build_cifar_10_res_net
import os
import tensorflow as tf

Using TensorFlow backend.


In [2]:
keras.__version__


'2.2.4'

In [3]:
tf.__version__

'1.12.0'

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
K.clear_session()
K.set_floatx('float32')
config = tf.ConfigProto(allow_soft_placement = True)
config.gpu_options.allow_growth = True

sess = tf.Session(config = config)
K.set_session(sess)

In [5]:
batch_size = 512
num_classes = 10
number_epoch = 24
workers =2
shuffle = True

In [6]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:
from data_utils import pad,normalise
x_train_raw = normalise(x_train.copy())
x_train = normalise(pad(x_train.copy().astype('uint8')).astype('float16'))
x_test = normalise(x_test)

In [8]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [9]:
y_train = y_train.astype('float16')
y_test = y_test.astype('float16')

In [10]:
y_train_int_label = y_train.copy().argmax(axis = 1)
y_test_int_label = y_test.copy().argmax(axis = 1)

num_train_samples = x_train.shape[0]
num_test_samples = x_test.shape[0]

In [11]:
selected_train_index = np.arange(num_train_samples)
dropped_train_index = np.array([],dtype = np.int64)
train_start_time = datetime.now()

In [12]:
def piecewise_linear_lr(current_epoch,number_epoch):
    return np.interp(current_epoch, [0, 5, number_epoch], [0, 0.4, 0])

In [13]:
model = build_cifar_10_res_net()

In [14]:
class LRReduce(keras.callbacks.Callback):
    def __init__(self,selected_x_train,number_epoch,e,batch_size):
        super(LRReduce, self).__init__()
        self.selected_x_train =selected_x_train
        self.number_epoch = number_epoch
        self.e = e
        self.batch_size = batch_size
    def on_batch_begin(self,batch,logs=None): 
        lr = piecewise_linear_lr(self.e+(batch/(self.selected_x_train.shape[0]/self.batch_size)),
                                    self.number_epoch)/self.batch_size
        K.set_value(self.model.optimizer.lr, lr)


In [15]:
optimizer_sgd = keras.optimizers.SGD(lr = 0.01,momentum=0.9,nesterov=True)
model.compile(
              loss='categorical_crossentropy', 
              optimizer=optimizer_sgd,
              metrics=['accuracy'])

In [16]:
threshold_prob = 0.00001
threshold_portion = 0.5

In [17]:
print_title = ['epoch','train time','train acc','test time','test acc','dropped samples','total time']
print(*(f'{item:>16s}' for item in print_title))

for e in range(number_epoch): 
    epoch_train_start_time = datetime.now()
    
    if shuffle:
        np.random.shuffle(selected_train_index)
        
    part_batch = selected_train_index.shape[0]%batch_size
    
    if part_batch!=0:
        part_batch_index = selected_train_index[-part_batch:].copy()
        selected_train_index = selected_train_index[0:-part_batch].copy()
        dropped_train_index = np.concatenate([part_batch_index,dropped_train_index],axis = 0)
        
    
    selected_x_train = x_train[selected_train_index].copy()
    selected_y_train = y_train[selected_train_index].copy() 
    
    if len(dropped_train_index)>0:
        dropped_x_train = x_train_raw[dropped_train_index].copy() 
        dropped_y_train = y_train[dropped_train_index].copy()
    
    train_generator = CIFAR10DataLoader(
            selected_x_train,
            selected_y_train,
            batch_size=batch_size,
            crop_size=32,
            cutout_size=8,
            is_train=True)
    
    history = GetPrediction()
   
        
    lr_reduce = LRReduce(selected_x_train,number_epoch,e,batch_size)
    model.fit_generator(
        train_generator,
        epochs=1,
        workers=workers,
        verbose=0,
        use_multiprocessing=False,
        validation_data=None,
        max_queue_size=10,
        shuffle = False,
        callbacks=[history,lr_reduce]
        )
  
    selected_prediction = history.prediction.copy()
    
    dropped_prediction = model.predict(dropped_x_train, batch_size=batch_size).copy()
    
    train_prediction = np.concatenate([selected_prediction,dropped_prediction],axis=0)
    train_prediction_index = np.concatenate([selected_train_index,dropped_train_index],axis=0)
    
    
    max_position = y_train_int_label[train_prediction_index]
    sequence_index = np.arange(num_train_samples)
    prob_error = 1.0 - train_prediction[(sequence_index,max_position)]
    
    selected_index_mask = (prob_error>threshold_prob)
    dropped_index_mask = (prob_error<=threshold_prob)
    
    selected_train_index = train_prediction_index[selected_index_mask]
    dropped_train_index = train_prediction_index[dropped_index_mask]
    
    dropped_portion = dropped_train_index.shape[0]/x_train.shape[0]
    
    if dropped_portion>threshold_portion:
        threshold_prob = threshold_prob/10
        if threshold_portion<0.9:
            threshold_portion=threshold_portion+0.1
        else:
            threshold_portion=threshold_portion+0.02
    
    train_accuracy = (train_prediction.argmax(axis=1) == max_position).sum()/num_train_samples
    

    epoch_train_end_time = datetime.now()
    epoch_training_time = (epoch_train_end_time-epoch_train_start_time).seconds
    
    
    test_prediction = model.predict(x_test, batch_size=batch_size).copy() 
    test_accuracy = (test_prediction.argmax(axis=1)==y_test_int_label).sum()/num_test_samples

    epoch_test_time = (datetime.now()-epoch_train_end_time).seconds
    total_time = (datetime.now()-train_start_time).seconds
    
    print_record = [e+1,epoch_training_time,train_accuracy,epoch_test_time,test_accuracy,len(dropped_train_index),total_time]

    print(*(f'{item:16.4f}' if isinstance(item, np.float) else f'{item:16}' for item in print_record))

           epoch       train time        train acc        test time         test acc  dropped samples       total time
               1               52           0.1563                3           0.1966                0               57
               2               42           0.2226                2           0.2786                0              102
               3               42           0.3079                2           0.3878                0              147
               4               41           0.4749                2           0.5741                0              191
               5               43           0.6158                2           0.6594                0              237
               6               41           0.6843                2           0.7274                0              281
               7               41           0.7230                2           0.7532                0              326
               8               41           0.75